In [2]:
from Utils.my_utils import *
from Recommenders.GraphBased.P3alphaRecommender import *
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
from Recommenders.NonPersonalizedRecommender import *
import os
from random import randint, seed

In [3]:
# setta anche il seed di random per sicurezza
seed = randint(1,1000)
print("Seed: " +str(seed))
np.random.seed(seed)
users, items, users_to_eval = readData()
URM, user_originalID_to_index, item_originalID_to_index, item_index_to_originalID = preProcess()
URM_train_full, URM_train, URM_test, URM_validation = splitURM(URM)

Seed: 63


In [4]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2615 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2189 (17.3%) Users that have less than 1 test interactions


In [5]:
output_folder_path = "result_experiments/"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

saved_folder_path = "result_best_models/"
if not os.path.exists(saved_folder_path):
    os.makedirs(saved_folder_path)
    
recommender_class = P3alphaRecommender    
n_cases = 10
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [6]:
runHyperparameterSearch_Collaborative(recommender_class,
       URM_train = URM_train,
       URM_train_last_test = URM_train_full,
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)

In [7]:
# load the model with all best parameters
recommender_object = P3alphaRecommender(URM_train_full)
recommender_object.load_model(output_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_best_model_last.zip")

ItemKNNCFRecommender: URM Detected 232 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 115 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender_tversky_best_model_last.zip'
ItemKNNCFRecommender: Loading complete
ItemKNNCFRecommender: URM Detected 232 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 115 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender_dice_best_model_last.zip'
ItemKNNCFRecommender: Loading complete
ItemKNNCFRecommender: URM Detected 232 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 115 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender_asymmetric_best_model_last.zip'
ItemKNNCFRecommender: Loading complete
ItemKNNCFRecommender: URM Detected 232 ( 1.8%) users with no interactions.
ItemKNNCFR

In [8]:
# load the TopPop model with all best parameters
recommender_object_top_pop = TopPop(URM_train_full)
recommender_object_top_pop.load_model(saved_folder_path, file_name = recommender_object_top_pop.RECOMMENDER_NAME + "_my_own_save.zip")

TopPopRecommender: URM Detected 232 ( 1.8%) users with no interactions.
TopPopRecommender: URM Detected 115 ( 0.5%) items with no interactions.
TopPopRecommender: Loading model from file 'result_best_models/TopPopRecommender_my_own_save.zip'
TopPopRecommender: Loading complete


In [9]:
# recommend items to users
rec_items_ind = []
for user_id in users_to_eval:
    if(user_id in users):
        # recommend users for which we already have some ratings
        rec_items_ind.append(recommender_object.recommend(user_originalID_to_index[user_id], cutoff=10))
    else:
        # recommend users with no stored rating
        rec_items_ind.append(recommender_object_top_pop.recommend(user_id, cutoff=10, remove_seen_flag=False))

In [10]:
# evaluate recommender
evaluate_algorithm(URM_test, recommender_object)

tversky


MAP = 0.0938
dice
MAP = 0.0881
asymmetric
MAP = 0.0946
jaccard
MAP = 0.0887
cosine
MAP = 0.0877


In [11]:
storeRecommendations(users_to_eval, rec_items_ind, item_index_to_originalID)

In [12]:
# save the model with all best parameters overwriting the old one
# recommender_object.save_model(saved_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_my_own_save.zip")